In [2]:
#import the packages
import cv2
import os
import re
import numpy
import dlib
import numpy as np
from skimage import io
import time

In [1]:
class VideoTransformer():

    def __init__(self, fps=30):
        self.fps = fps

    def video_to_frame(self, path_video, dir_frames):

        vidcap = cv2.VideoCapture(path_video)
        success,image = vidcap.read()
        count = 0
        while success:
            cv2.imwrite(dir_frames + "/frame_" + str(count).zfill(10) + ".jpg", image)     # save frame as JPEG file      
            success,image = vidcap.read()
            #print('Read a new frame: ', success)
            count += 1


    def frame_to_video(self, dir_frames, path_video, fps):
        num_img = []
        images = []
        for img in os.listdir(dir_frames):
            if img.endswith(".png") or img.endswith(".jpg"):
                # using List comprehension + isdigit() +split() 
                # getting numbers from string  
                data = img.split("_")
                data = data[1].split(".")
                num = data[0]
                #print(num)
                num_img.append(num)
                images.append(img)

        sorted_numbers = numpy.argsort(num_img)
        images_sorted = []
        #print(images)
        for i in range(len(images)):
            images_sorted.append(images[sorted_numbers[i]])

        frame = cv2.imread(os.path.join(dir_frames, images_sorted[0]))
        height, width, layers = frame.shape
        fourrc = cv2.VideoWriter_fourcc(*'MP4V')
        video = cv2.VideoWriter(path_video, fourrc, fps, (width, height))

        for image in images_sorted:
            frame = cv2.imread(os.path.join(dir_frames, image))
            video.write(frame)

        cv2.destroyAllWindows()
        video.release()

In [2]:
# Set parameters

PATH_VID = "/home/john/dlib_files/input/sample1.mp4"# location video 
DIR_FRAMES = "/home/john/dlib_files/frames"# directory to store frames (will be created)
PATH_VID_AN = "/home/john/dlib_files/output/sample1_output_0.35_68.mp4"# name of annotated video
DIR_FRAMES_AN = "/home/john/dlib_files/labeled_frames/"# directory to store annotated frames (will be created)
VID_FPS = 30

print("running")

# Create objects
# img_annotator = ImageAnnotator(PATH_CRED)
video_transf = VideoTransformer(VID_FPS)

# create folder for frames
try:
    os.mkdir(DIR_FRAMES)
except:
    print("Folder " + DIR_FRAMES + " already exists")
    for filename in os.listdir(DIR_FRAMES):
        file_path = os.path.join(DIR_FRAMES, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

try:
    os.mkdir(DIR_FRAMES_AN)
except:
    print("Folder " + DIR_FRAMES_AN + " already exists")
    for filename in os.listdir(DIR_FRAMES_AN):
        file_path = os.path.join(DIR_FRAMES_AN, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# Turn video into frames
print("Turn video into frames")
video_transf.video_to_frame(PATH_VID, DIR_FRAMES)



running
Folder /home/john/dlib_files/frames already exists
Folder /home/john/dlib_files/labeled_frames/ already exists
Turn video into frames


In [3]:
data_dir = os.path.expanduser('/home/john/dlib_files')
faces_folder_path = data_dir + '/unique_faces/'

# Globals
face_detector = dlib.get_frontal_face_detector()
shape_predictor = dlib.shape_predictor(data_dir + '/shape_predictor_68_face_landmarks.dat')
# shape_predictor = dlib.shape_predictor(data_dir + '/shape_predictor_5_face_landmarks.dat')
face_recognition_model = dlib.face_recognition_model_v1(data_dir + '/dlib_face_recognition_resnet_model_v1.dat')


def get_face_encodings(face):
    bounds = face_detector(face, 1)
    faces_landmarks = [shape_predictor(face, face_bounds) for face_bounds in bounds]
    return [np.array(face_recognition_model.compute_face_descriptor(face, face_pose, 1)) for face_pose in faces_landmarks]


def get_face_matches(known_faces, face):
    return np.linalg.norm(known_faces - face, axis=1)


def find_match(known_faces, person_names, face):
    matches = get_face_matches(known_faces, face) # get a list of True/False
    min_index = matches.argmin()
    min_value = matches[min_index]
    if min_value < 0.50:
        return person_names[min_index]+"! ({0:.2f})".format(min_value)
    if min_value < 0.55:
        return person_names[min_index]+" ({0:.2f})".format(min_value)
    if min_value < 0.60:
        return person_names[min_index]+"?"+" ({0:.2f})".format(min_value)
    return 'Not Found'


def load_face_encodings(faces_folder_path):
    image_filenames = filter(lambda x: x.endswith('.jpg'), os.listdir(faces_folder_path))
    image_filenames = sorted(image_filenames)
    person_names = [x[:-4] for x in image_filenames]

    full_paths_to_images = [faces_folder_path + x for x in image_filenames]
    face_encodings = []

    win = dlib.image_window()

    for path_to_image in full_paths_to_images:
        face = io.imread(path_to_image)

        faces_bounds = face_detector(face, 1)

#         if len(faces_bounds) != 1:
#             print("Expected one and only one face per image: " + path_to_image + " - it has " + str(len(faces_bounds)))
#             exit()
        try:
            face_bounds = faces_bounds[0]
        except IndexError:
            pass
        face_landmarks = shape_predictor(face, face_bounds)
        
        face_encoding = np.array(face_recognition_model.compute_face_descriptor(face, face_landmarks, 1))

        win.clear_overlay()
        win.set_image(face)
        win.add_overlay(face_bounds)
        win.add_overlay(face_landmarks)
        face_encodings.append(face_encoding)

#         print(face_encoding)

        #dlib.hit_enter_to_continue()
    return face_encodings, person_names



In [4]:
def recognize_faces_in_video(face_encodings, person_names):
    
    face_cascade = cv2.CascadeClassifier(data_dir + '/haarcascade_frontalface_default.xml')
#     face_cascade = cv2.CascadeClassifier('/home/ml/Desktop/opencv-master/data/haarcascades_cuda/haarcascade_frontalface_default.xml')
    unknown_counter = 0
    font = cv2.FONT_HERSHEY_SIMPLEX
    for image in os.listdir("/home/john/dlib_files/frames"):
        image_path = os.path.join("/home/john/dlib_files/frames",image)
        image_name = image_path.split('/')[-1].strip('.jpg')
        frame = cv2.imread(image_path)
#         img = frame
        height, width, channels = frame.shape
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        # Loop through all the faces detected 
        face_encodings_in_image = []
        for (x, y, w, h) in faces:
            x1 = x
            y1 = y
            x2 = x+w
            y2 = y+h
#             cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face = frame[max(0, y1):min(height, y2), max(0, x1):min(width, x2)] 
            face_encodings_in_image = get_face_encodings(face)
            if len(face_encodings_in_image) >0:
                img = cv2.rectangle(frame,(x1, y1),(x2, y2),(0,255,0),2)
                match = find_match(face_encodings, person_names, face_encodings_in_image[0])
                cv2.putText(frame, str(match), (x+5, y-15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
                name = "/home/john/dlib_files/labeled_frames/"+ str(image_name) + '.jpg'
            else:
                cv2.rectangle(frame,(x1, y1),(x2, y2), (0, 0, 255), 2)
                name = "/home/john/dlib_files/labeled_frames/"+ str(image_name) + '.jpg'
                cv2.imwrite(name, frame)
        name = "/home/john/dlib_files/labeled_frames/"+ str(image_name) + '.jpg'
        cv2.imwrite(name, frame)
        key = cv2.waitKey(100)
#         cv2.imshow("Face Recognizer", frame)

        if key == 27: # exit on ESC
            break
#     vc.release()
    cv2.destroyAllWindows()

In [1]:
t1 = time.perf_counter()

face_encodings, person_names = load_face_encodings(faces_folder_path)
recognize_faces_in_video(face_encodings, person_names)

t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

In [6]:
# transform frames to video
print("Transform frames to video")
video_transf.frame_to_video(DIR_FRAMES_AN, PATH_VID_AN,VID_FPS)
print("Done!")

Get informations from images
Transform frames to video
Done!
